In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

     |████████████████████████████████| 1.7 MB 7.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# !pip install google-auth>=1.21.1
# !pip install httplib2==0.17.0
!pip install tflite-support
# !pip install -U tfx
# !pip install python-snappy

     |████████████████████████████████| 1.1 MB 7.8 MB/s            
     |████████████████████████████████| 208 kB 87.9 MB/s            
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


In [ ]:
pip install tensorflow_model_analysis==0.35.0

     |████████████████████████████████| 1.8 MB 9.0 MB/s            
     |████████████████████████████████| 9.8 MB 54.2 MB/s            
     |████████████████████████████████| 791 kB 68.5 MB/s            
     |████████████████████████████████| 19.1 MB 35.9 MB/s            
     |████████████████████████████████| 829 kB 55.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 2.3 MB 46.8 MB/s            
     |████████████████████████████████| 249 kB 73.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 62 kB 1.1 MB/s             
     |████████████████████████████████| 151 kB 99.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 173 kB 82.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 255 kB 91.9 MB/s            
     |████████████████████████████████| 169 kB 109.2 MB/s            
     |██

In [ ]:
pip install tfx

     |████████████████████████████████| 2.4 MB 7.5 MB/s            
     |████████████████████████████████| 1.6 MB 43.5 MB/s            
     |████████████████████████████████| 49 kB 7.5 MB/s             
     |████████████████████████████████| 1.4 MB 62.8 MB/s            
     |████████████████████████████████| 413 kB 90.6 MB/s            
     |████████████████████████████████| 98 kB 8.5 MB/s             
     |████████████████████████████████| 6.6 MB 10.8 MB/s            
     |████████████████████████████████| 203 kB 70.0 MB/s            
     |████████████████████████████████| 147 kB 58.4 MB/s            
     |████████████████████████████████| 1.2 MB 49.0 MB/s            
     |████████████████████████████████| 40 kB 7.2 MB/s             
     |████████████████████████████████| 636 kB 76.6 MB/s            
     |████████████████████████████████| 458.3 MB 12 kB/s             
     |████████████████████████████████| 1.7 MB 56.7 MB/s            
     |███████████████████████████████

In [ ]:
pip install tensorflow==2.6.0

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx

from tfx.components.example_gen.import_example_gen.component import ImportExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
# from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform

from tfx.dsl.experimental import latest_blessed_model_resolver

from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

# from tfx.utils.dsl_utils import external_input

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))
print('TFMA bersion:{}'.format(tfma.__version__))

TensorFlow version: 2.6.0
TFX version: 1.4.0
TFMA bersion:0.35.0


In [ ]:
!mkdir ./data


In [ ]:
!mkdir ./data/tfrecords


In [ ]:
!cp -r /content/drive/Shareddrives/297_project/plant-pathology/images2.tfrecords ./data/tfrecords/

In [ ]:
# !cp -r /content/drive/Shareddrives/297_project/plant-pathology/tfrecord/old/plant_patho_test.tfrecord  ./data/test/

In [ ]:
!ls ./data

tfrecords


In [ ]:
_pipeline_name = 'plantpatho_native_keras'

# root directory for TFX pip package installation.
_plantpatho_root = "."
print("Root dir: " + _plantpatho_root)

_data_root = os.path.join(_plantpatho_root, 'data')
# print(_data_root)
# Serving path
_serving_model_dir = os.path.join('./serving_model')
# print("TF Serving dir: ", _serving_model_dir)

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

# Set up utils file
# _module_file = os.path.join(_plantpatho_root, 'plant_utils_native_keras.py')
_trainer_module_file = os.path.join(os.getcwd(), 'trainer2.py')

# input_config = example_gen_pb2.Input(splits=[
#     example_gen_pb2.Input.Split(name='train', pattern='train/*'),
#     example_gen_pb2.Input.Split(name='eval', pattern='test/*')
# ])

_tfx_root = os.path.join(".", 'tfx_output')

_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)


metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name, 'metadata.db')
# print(_tfx_root,_pipeline_root,metadata_path )

_labels_path = os.path.join(_data_root, 'labels.txt')

Root dir: .


In [ ]:
_beam_pipeline_args = [
    '--direct_running_mode=multi_processing',
    # 0 means auto-detect based on on the number of CPUs available
    # during execution time.
    '--direct_num_workers=0',
]

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext(_plantpatho_root)


# ExampleGen

In [ ]:
#example_gen = ImportExampleGen(input_base="./data/", input_config=input_config)
example_gen = ImportExampleGen(input_base="./data/tfrecords")
context.run(example_gen)

INFO:absl:Running driver for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for ImportExampleGen
INFO:absl:Generating examples.


INFO:absl:Reading input TFRecord data ./data/tfrecords/*.
INFO:absl:Examples generated.
INFO:absl:Running publisher for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/tmp/tfx-.-gaok3r7j/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:1153782297,xor_checksum:1637818595,sum_checksum:1637818595"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Get the url, the list of files, iterating over 1 record

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-.-gaok3r7j/ImportExampleGen/examples/1


In [ ]:
#Getting Train UrL
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

In [ ]:
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
# Iterate over the first 1 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "image"
    value {
      bytes_list {
        value: "\253\252*?\263\2622?\202\201\001?\024\333\035?\331A\'?\032\005\350>|\305\022?0( ?\333\254\325>v\220\n?\207\236\030?\244\322\306>+\277\016?\327% ?/\246\323>\016w\023?(\276!?\337\334\334>\277\253\021?\363\274\"?f\021\335>L-\372>\270(\017?\006\351\265>y\377\364>Ut\013?@\200\256>Y\355\376>\276\207\020?\022\247\270>\212\314\014?\234\335\035?\317R\323>\203\353\022?\225\374#?\302\220\337>m\264\005?\177\305\026?\225\"\305>z\333\361>\315\376\t?3\225\253>\277\314\357>q\367\010?x\206\251>\312C\351>\032\257\002?\215\334\236>\342`\335>\220\016\374>\277=\234>FO\314>\200X\354>\312n\222>\313L\266>\207\261\324>7\222|>\017T\301>l\202\331>\274\351\203>\021[\274>\354j\344>[S\211>\360\316\230>w\335\314>\336#]>g\000\221>\363F\304>\344\320^>|\237\212>\262\325\300>\271\376T>\262\201\204>\277\276\276>\373\311L>\203\202\202>\320E\272>\334QF>z\224\200>\260\312\266>\265\350@>\016\206\202>D\274\270>\334\313D>P\204\205>\206\272\2

# StatisticsGen

In [ ]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-.-gaok3r7j/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-.-gaok3r7j/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "/tmp/tfx-.-gaok3r7j/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context.show(statistics_gen.outputs['statistics'])

Data statistics are a little lackluster for this image dataset

#SchemaGen

In [ ]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-.-gaok3r7j/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/tmp/tfx-.-gaok3r7j/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'image',BYTES,required,single,-
'label',INT,required,single,-


# Example Validator

In [ ]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-.-gaok3r7j/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-.-gaok3r7j/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "/tmp/tfx-.-gaok3r7j/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context.show(example_validator.outputs['anomalies'])

#Trainer

In [ ]:
from typing import List

import absl
import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options


def transformed_name(key):
  return key + '_xf'

#Set up paths and the create a trainer file name.
_trainer_file_name = 'trainer2.py'





In [ ]:
%%writefile {_trainer_file_name}
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from typing import List, Text
import absl
import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
import keras
import keras.backend as K
from keras.utils import generic_utils
from PIL import Image
import PIL
from keras.models import Sequential, Model, load_model, model_from_json
from keras import layers
from keras.layers import Dense, Flatten, Dropout, Activation, LeakyReLU, Reshape, Concatenate, Input
from keras.layers import Conv2D, UpSampling2D, Conv2DTranspose, MaxPool2D
from tensorflow.keras.layers import BatchNormalization
from keras import optimizers, losses
from copy import deepcopy
from tfx.components.trainer.fn_args_utils import FnArgs
from IPython.display import display, Image, Markdown, SVG
from keras.utils.vis_utils import model_to_dot
import itertools
import numpy as np
PATH_FOR_100PCT="gs://simclr-checkpoints-tf2/simclrv2/finetuned_100pct/r50_1x_sk0/saved_model/"
 
EETA_DEFAULT = 0.001     
MOMENTUM = 0.9            
# BATCH_SIZE = 64      
LEARNING_RATE = 0.1      
WEIGHT_DECAY = 0.0         
TOTAL_ITERATIONS = 20   
num_train_classes=4

# LARS Optimizer

class LARSOptimizer(tf.keras.optimizers.Optimizer):
  """Layer-wise Adaptive Rate Scaling for large batch training.

  Introduced by "Large Batch Training of Convolutional Networks" by Y. You,
  I. Gitman, and B. Ginsburg. (https://arxiv.org/abs/1708.03888)
  """

  def __init__(self,
               learning_rate,
               momentum=MOMENTUM,
               use_nesterov=False,
               weight_decay=0.0,
               exclude_from_weight_decay=None,
               exclude_from_layer_adaptation=None,
               classic_momentum=True,
               eeta=EETA_DEFAULT,
               name="LARSOptimizer"):
    """Constructs a LARSOptimizer.

    Args:
      learning_rate: A `float` for learning rate.
      momentum: A `float` for momentum.
      use_nesterov: A 'Boolean' for whether to use nesterov momentum.
      weight_decay: A `float` for weight decay.
      exclude_from_weight_decay: A list of `string` for variable screening, if
          any of the string appears in a variable's name, the variable will be
          excluded for computing weight decay. For example, one could specify
          the list like ['batch_normalization', 'bias'] to exclude BN and bias
          from weight decay.
      exclude_from_layer_adaptation: Similar to exclude_from_weight_decay, but
          for layer adaptation. If it is None, it will be defaulted the same as
          exclude_from_weight_decay.
      classic_momentum: A `boolean` for whether to use classic (or popular)
          momentum. The learning rate is applied during momeuntum update in
          classic momentum, but after momentum for popular momentum.
      eeta: A `float` for scaling of learning rate when computing trust ratio.
      name: The name for the scope.
    """
    super(LARSOptimizer, self).__init__(name)

    self._set_hyper("learning_rate", learning_rate)
    self.momentum = momentum
    self.weight_decay = weight_decay
    self.use_nesterov = use_nesterov
    self.classic_momentum = classic_momentum
    self.eeta = eeta
    self.exclude_from_weight_decay = exclude_from_weight_decay
    # exclude_from_layer_adaptation is set to exclude_from_weight_decay if the
    # arg is None.
    if exclude_from_layer_adaptation:
      self.exclude_from_layer_adaptation = exclude_from_layer_adaptation
    else:
      self.exclude_from_layer_adaptation = exclude_from_weight_decay

  def _create_slots(self, var_list):
    for v in var_list:
      self.add_slot(v, "Momentum")

  def _resource_apply_dense(self, grad, param, apply_state=None):
    if grad is None or param is None:
      return tf.no_op()

    var_device, var_dtype = param.device, param.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype)) or
                    self._fallback_apply_state(var_device, var_dtype))
    learning_rate = coefficients["lr_t"]

    param_name = param.name

    v = self.get_slot(param, "Momentum")

    if self._use_weight_decay(param_name):
      grad += self.weight_decay * param

    if self.classic_momentum:
      trust_ratio = 1.0
      if self._do_layer_adaptation(param_name):
        w_norm = tf.norm(param, ord=2)
        g_norm = tf.norm(grad, ord=2)
        trust_ratio = tf.where(
            tf.greater(w_norm, 0),
            tf.where(tf.greater(g_norm, 0), (self.eeta * w_norm / g_norm), 1.0),
            1.0)
      scaled_lr = learning_rate * trust_ratio

      next_v = tf.multiply(self.momentum, v) + scaled_lr * grad
      if self.use_nesterov:
        update = tf.multiply(self.momentum, next_v) + scaled_lr * grad
      else:
        update = next_v
      next_param = param - update
    else:
      next_v = tf.multiply(self.momentum, v) + grad
      if self.use_nesterov:
        update = tf.multiply(self.momentum, next_v) + grad
      else:
        update = next_v

      trust_ratio = 1.0
      if self._do_layer_adaptation(param_name):
        w_norm = tf.norm(param, ord=2)
        v_norm = tf.norm(update, ord=2)
        trust_ratio = tf.where(
            tf.greater(w_norm, 0),
            tf.where(tf.greater(v_norm, 0), (self.eeta * w_norm / v_norm), 1.0),
            1.0)
      scaled_lr = trust_ratio * learning_rate
      next_param = param - scaled_lr * update

    return tf.group(*[
        param.assign(next_param, use_locking=False),
        v.assign(next_v, use_locking=False)
    ])

  def _use_weight_decay(self, param_name):
    """Whether to use L2 weight decay for `param_name`."""
    if not self.weight_decay:
      return False
    if self.exclude_from_weight_decay:
      for r in self.exclude_from_weight_decay:
        if re.search(r, param_name) is not None:
          return False
    return True

  def _do_layer_adaptation(self, param_name):
    """Whether to do layer-wise learning rate adaptation for `param_name`."""
    if self.exclude_from_layer_adaptation:
      for r in self.exclude_from_layer_adaptation:
        if re.search(r, param_name) is not None:
          return False
    return True

  def get_config(self):
    config = super(LARSOptimizer, self).get_config()
    config.update({
        "learning_rate": self._serialize_hyperparameter("learning_rate"),
        "momentum": self.momentum,
        "classic_momentum": self.classic_momentum,
        "weight_decay": self.weight_decay,
        "eeta": self.eeta,
        "use_nesterov": self.use_nesterov,
    })
    return config

class Model(tf.keras.Model):
  def __init__(self, path):
    super(Model, self).__init__()
    self.saved_model = tf.saved_model.load(path)
    self.dense_layer = tf.keras.layers.Dense(units=num_train_classes, name="head_supervised_new")
    self.optimizer = LARSOptimizer(
      LEARNING_RATE,
      momentum=MOMENTUM,
      weight_decay=WEIGHT_DECAY,
      exclude_from_weight_decay=['batch_normalization', 'bias', 'head_supervised'])

  def call(self, x):
    with tf.GradientTape() as tape:
      
      print(">>>>>>>>>>>>>>>>>>>",x[0])
      #outputs = self.saved_model(x['image'], trainable=False)
      outputs = self.saved_model(x[0], trainable=False)
      print(outputs)
      logits_t = self.dense_layer(outputs['final_avg_pool'])
      labels = tf.one_hot(x[1], num_train_classes)
      print("labels<<<<<<<",labels)
      print("logits",logits_t)
      loss_t = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf.one_hot(x[1], num_train_classes), logits=logits_t))
      dense_layer_weights = self.dense_layer.trainable_weights
      print('Variables to train:', dense_layer_weights)
      grads = tape.gradient(loss_t, dense_layer_weights)
      self.optimizer.apply_gradients(zip(grads, dense_layer_weights))
    return loss_t, x[0], logits_t, x[1]

model = Model(PATH_FOR_100PCT)

@tf.function
def train_step(x):
  return model(x)
#title TensorBoard Logger
class Logger(object):
    """Logging in tensorboard without tensorflow ops."""

    def __init__(self, log_dir):
        """Creates a summary writer logging to log_dir."""
        self.writer = tf.summary.create_file_writer(log_dir)

    def log_scalar(self, tag, value, step):
        """Log a scalar variable."""
        with self.writer.as_default():
              tf.summary.scalar(tag, value,step=step)
              self.writer.flush()

#fine tune the linear layer
def finetune(train_dataset,validation_dataset,TOTAL_ITERATIONS=2,t_exp=""):
  y_true,y_pred = [], []

  # Train and Test Iterators
  train_iterator = iter(train_dataset)
  test_iterator = iter(validation_dataset)

  # Initialize TensorBoard Logger
  l = Logger("logs/scalars"+t_exp)

  # Training Loop
  for it in range(TOTAL_ITERATIONS):
    # Iterate over train images
    x = next(train_iterator)
    print("tamanna",x)
   
    train_loss, train_image, train_logits, train_labels = train_step(x)
    train_logits = train_logits.numpy()
    train_labels = train_labels.numpy()
    pred_train = train_logits.argmax(-1)
    num_train_correct = np.sum(pred_train == train_labels)
    y_true = np.append(y_true,train_labels)
    y_pred = np.append(y_pred,pred_train)
    total_train = train_labels.size
    l.log_scalar("Train Loss",train_loss,it+1)
    l.log_scalar("Train Top 1 Accuracy:",num_train_correct/float(total_train),it+1)
    print("[Iter {}] Loss: {} Top1 Accuracy: {}".format(it+1, train_loss, num_train_correct/float(total_train)))

  print("----------Evaluate the model----------") 
  for it in range(6): 
    # Iterate over test images
    x = next(test_iterator)
    test_loss, test_image, test_logits, test_labels = train_step(x)
    # print(test_loss, test_image, test_logits, test_labels)
    test_logits = test_logits.numpy()
    test_labels = test_labels.numpy()
    pred_test = test_logits.argmax(-1)
    num_test_correct = np.sum(pred_test == test_labels)
    total_test = test_labels.size
    l.log_scalar("Test Loss",test_loss,it+1)
    l.log_scalar("Test Top 1 Accuracy:",num_test_correct/float(total_test),it+1)
    print("[Iter {}] test_loss: {} test_top1_accuracy: {}".format(it+1, test_loss, num_test_correct/float(total_test)))
  
  print("----------Results----------")
  print(classification_report(y_true, y_pred, target_names=plant_patho_labels))
  # Plot the images and predictions
  fig, axes = plt.subplots(5, 1, figsize=(15, 15))
  for i in range(5):
    axes[i].imshow(train_image[i])
    true_text = plant_patho_labels[train_labels[i]]
    pred_text = plant_patho_labels[pred_train[i]]
    axes[i].axis('off')
    axes[i].text(256, 128, 'Truth: ' + true_text + '\n' + 'Pred: ' + pred_text)

# def feature_engg(features, label):
#   #Add new features
#   # feature = _preprocess(features)
#   image = tf.io.decode_raw(feature['image'], tf.uint8)
#   image=tf.cast(x=images, dtype=tf.float32)
#   image = tf.cast(image, tf.float32) * (1. / 255)
#   label = tf.cast(features['label'], tf.int32)

  # return(features, label)
# make_input function to be called for the trainer module
def make_input_fn(data_root, mode, vnum_epochs = None, batch_size=4):
    def decode_tfr(serialized_example):
      # define a parser
      features = tf.io.parse_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64)
        })
      image = tf.io.decode_raw(features['image'], tf.uint8)
      image = tf.reshape(image, [-1, 256, 256, 3])
      image=tf.cast(image, dtype=tf.float32)* (1. / 255)
      print("image.shape",image.shape)
      label = tf.cast(features['label'], tf.int32)
     


      return image,label
# input function for the trainer module file
    def _input_fn(v_test=False):
      # Get the list of files in this directory (all compressed TFRecord files)
      tfrecord_filenames = tf.io.gfile.glob(data_root)

      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

      if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = vnum_epochs # indefinitely
      else:
        num_epochs = 1 # end-of-input after this

      dataset = dataset.batch(batch_size)
      dataset = dataset.prefetch(buffer_size = batch_size)

      #Convert TFRecord data to dict
      dataset = dataset.map(decode_tfr)

      #Feature engineering
      # dataset = dataset.map(feature_engg)

      if mode == tf.estimator.ModeKeys.TRAIN:
          num_epochs = vnum_epochs # indefinitely
          dataset = dataset.shuffle(buffer_size = batch_size)
      else:
          num_epochs = 1 # end-of-input after this

      dataset = dataset.repeat(num_epochs)       
      
      #Begins - Uncomment for testing only -----------------------------------------------------<
      if v_test == True:
        print(next(dataset.__iter__()))
        
      #End - Uncomment for testing only -----------------------------------------------------<
      print("DATA SIZE")
      print(dataset.cardinality)
      return dataset
    return _input_fn
## function to save the model
def save_model(model, model_save_path):
  @tf.function
  def serving(image):
      ##Feature engineering

      payload = {
          'image': image
      }
      
      ## Prediction
      ##IF THERE IS AN ERROR IN NUMBER OF PARAMS PASSED HERE OR DATA TYPE THEN IT GIVES ERROR, "COULDN'T COMPUTE OUTPUT TENSOR"
      predictions = model(payload)
      return predictions

  serving = serving.get_concrete_function(image=tf.TensorSpec([None, 256,256,3], dtype=tf.uint8, name='image')
                                          )
# save the model on the path
  # version = "1"  #{'serving_default': call_output}
  tf.saved_model.save(
      model,
      model_save_path + "/",
      signatures=serving)
  
  
def run_fn(fn_args: FnArgs):
  """Train the model based on given args.
  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  # print("Starting Training!!!!!!!!!!!!!!")
  # Getting custom arguments
  # batch_size = fn_args.custom_config['batch_size']
  # data_size = fn_args.custom_config['data_size']



#initializing the train dataset and the validation dataset
  train_dataset = make_input_fn(data_root = fn_args.train_files,
                      mode = tf.estimator.ModeKeys.TRAIN,
                      batch_size=4)()

  validation_dataset = make_input_fn(data_root = fn_args.eval_files,
                      mode = tf.estimator.ModeKeys.EVAL,
                      batch_size=4)()    

  mirrored_strategy = tf.distribute.MirroredStrategy()
  with mirrored_strategy.scope():

    finetune(train_dataset,validation_dataset,TOTAL_ITERATIONS=5,t_exp="_100pct")

 
# Write logs to path
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=fn_args.model_run_dir, update_freq='batch')
 

  # show_images(train_dataset)
  
  



  save_model(generator,fn_args.serving_model_dir)
  print('serving model dir',fn_args.serving_model_dir)

Writing trainer2.py


In [ ]:
#Test the input function
import trainer2 

#Test dataset read + Feat Engg function's - output's CSV + Feature engg columns
eval_file = os.path.join(example_gen.outputs['examples'].get()[0].uri, '/tmp/tfx-.-cx6be3df/ImportExampleGen/examples/3/Split-eval/*')
fn_d = trainer2.make_input_fn(data_root = eval_file,
                    mode = tf.estimator.ModeKeys.EVAL,
                    # vnum_epochs = 1,
                    batch_size = 4)()

image.shape (None, 256, 256, 3)
DATA SIZE
<bound method DatasetV2.cardinality of <RepeatDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>>


##Feature Engg

In [ ]:
from tfx.components.trainer.executor import GenericExecutor
from tfx.components.base import executor_spec

In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     trainer_module_file: str,
                     serving_model_dir: str, 
                     metadata_path: str,
                     labels_path:str,
                     beam_pipeline_args) -> pipeline.Pipeline:
  # Brings data into the pipeline.
  example_gen = ImportExampleGen(input_base="./data/tfrecords")


  # Computes statistics over data for visualization and example validation.
  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  # Generates schema based on statistics files.
  schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  # Performs anomaly detection based on statistics and data schema.
  example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema'])
  
  # transform = Transform(
  #   examples=example_gen.outputs['examples'],
  #   schema=schema_gen.outputs['schema'],
  #   module_file=_module_file,)

# function definition to create the trainer component
  def _create_trainer(module_file, component_id):
    return Trainer(custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(num_steps=5),
        eval_args=trainer_pb2.EvalArgs(num_steps=5),
        custom_config={'labels_path': labels_path})

  # Uses user-provided Python function that trains a Keras model.
  trainer = _create_trainer(trainer_module_file, 'Trainer.mnist')

  


  # Uses TFMA to compute evaluation statistics over features of a model and
  # performs quality validation of a candidate model.
  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='label')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  # threshold=tfma.config.MetricThreshold(
                  #     value_threshold=tfma.GenericValueThreshold(
                  #         lower_bound={'value': 0.8}))
                  )
          ])
      ])





  # Uses TFMA to compute the evaluation statistics over features of a model.
  evaluator = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      eval_config=eval_config).with_id('Evaluator.mnist')

 

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir))).with_id('Pusher.mnist')

 

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=[
          example_gen,
          statistics_gen,
          schema_gen,
          example_validator,
          trainer,
          pusher
          
      ],
      enable_cache=True,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(
          metadata_path),
      beam_pipeline_args=beam_pipeline_args)


In [ ]:
if __name__ == '__main__':
  absl.logging.set_verbosity(absl.logging.INFO)
  BeamDagRunner().run(
      _create_pipeline(
          pipeline_name=_pipeline_name,
          pipeline_root=_plantpatho_root,
          data_root='./data/tfrecords',
          trainer_module_file=_trainer_module_file,
          serving_model_dir=_serving_model_dir,
          metadata_path=metadata_path,
          labels_path=_labels_path,
          beam_pipeline_args=_beam_pipeline_args,
          # training_params={'batch_size': 4, "data_size":1}
          ))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/content/trainer2.py' (including modules: ['trainer2']).
INFO:absl:User module package has hash fingerprint version 1986f34c8310bd9c89455075d32924b8789325d72ebec9360e0ed1d82123b161.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpk68zakwi/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp8ss9ra_t', '--dist-dir', '/tmp/tmpv1pdnax7']
INFO:absl:Successfully built user code wheel distribution at './_wheels/tfx_user_code_Trainer-0.0+1986f34c8310bd9c89455075d32924b8789325d72ebec9360e0ed1d82123b161-py3-none-any.whl'; target user module is 'trainer2'.
INFO:absl:Full user module path is 'trainer2@./_wheels/tfx_user_code_Trainer-0.0+1986f34c8310bd9c89455075d32924b8789325d72ebec9360e0ed1d82123b161-py3-none-any.whl'
INFO:absl:Using deploym

image.shape (None, 256, 256, 3)
DATA SIZE
<bound method DatasetV2.cardinality of <RepeatDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>>
image.shape (None, 256, 256, 3)
DATA SIZE
<bound method DatasetV2.cardinality of <RepeatDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>>
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
tamanna (<tf.Tensor: shape=(16, 256, 256, 3), dtype=float32, numpy=
array([[[[0.64705884, 0.6431373 , 0.6431373 ],
         [0.24313727, 0.5529412 , 0.54901963],
         [0.04705883, 0.24705884, 0.5058824 ],
         ...,
         [0.24313727, 0.9490197 , 0.5686275 ],
         [0.01568628, 0.24705884, 0.9843138 ],
         [0.60784316, 0.05490196, 0.24705884]],

        [[0.64705884, 0.6431373 , 0.6431373 ],
         [0.24313727, 0.5137255 , 0.50980395],
         [0.00784314, 0.24705884, 0.5529412 ],
         ...,
         [0.24313727, 0.16470589, 0.90588

INFO:absl:MetadataStore with DB connection initialized
ERROR:absl:Execution 17 failed.
INFO:absl:Cleaning up stateless execution info.


{'initial_max_pool': <tf.Tensor 'model_1/StatefulPartitionedCall:6' shape=(16, 64, 64, 64) dtype=float32>, 'block_group2': <tf.Tensor 'model_1/StatefulPartitionedCall:1' shape=(16, 32, 32, 512) dtype=float32>, 'final_avg_pool': <tf.Tensor 'model_1/StatefulPartitionedCall:4' shape=(16, 2048) dtype=float32>, 'logits_sup': <tf.Tensor 'model_1/StatefulPartitionedCall:7' shape=(16, 1000) dtype=float32>, 'block_group1': <tf.Tensor 'model_1/StatefulPartitionedCall:0' shape=(16, 64, 64, 256) dtype=float32>, 'initial_conv': <tf.Tensor 'model_1/StatefulPartitionedCall:5' shape=(16, 128, 128, 64) dtype=float32>, 'block_group3': <tf.Tensor 'model_1/StatefulPartitionedCall:2' shape=(16, 16, 16, 1024) dtype=float32>, 'block_group4': <tf.Tensor 'model_1/StatefulPartitionedCall:3' shape=(16, 8, 8, 2048) dtype=float32>}
labels<<<<<<< Tensor("model_1/one_hot:0", shape=(4, 4), dtype=float32)
logits Tensor("model_1/head_supervised_new/BiasAdd:0", shape=(16, 4), dtype=float32)


ValueError: ignored